In [1]:
from ultralytics import YOLO
import numpy as np
import cv2
import pickle
import catcher as CT
import torch
import SaveSystem_by_grip
import SaveSystem_by_environment

print(torch.cuda.is_available())
Save_2_grip = SaveSystem_by_grip.save_system()
Save_2_environ = SaveSystem_by_environment.save_system()

surface_model = YOLO("cube_surface_seg2.pt")
cube_model = YOLO("yolov8n-seg-custom.pt")

with open('./hand_matrix/calibration.pkl', 'rb') as file:
    camera_matrix, dist_coeff = pickle.load(file)
# print(camera_matrix, dist_coeff)
# with open('./hand_matrix/camMatrix.npy', 'rb') as file:
#     m_camera_matrix = np.load(file)
# with open('./hand_matrix/distCoef.npy', 'rb') as file:
#     m_dist_coeff = np.load(file)
# print(m_camera_matrix, m_dist_coeff)
    
CT = CT.block_detect(surface_model=surface_model, cube_model=cube_model)

True


In [ ]:
#HSV
import cv2
def empty(v):
    pass

block = cv2.imread('./images/img5.png')
block = cv2.resize(block, (0, 0), fx=.7, fy=.7)

cv2.namedWindow('TrackBar')
cv2.resizeWindow('TrackBar', 640, 320)

cv2.createTrackbar('Hue Min', 'TrackBar', 0, 179, empty)
cv2.createTrackbar('Hue Max', 'TrackBar', 179, 179, empty)
cv2.createTrackbar('Sat Min', 'TrackBar', 0, 255, empty)
cv2.createTrackbar('Sat Max', 'TrackBar', 255, 255, empty)
cv2.createTrackbar('Val Min', 'TrackBar', 0, 255, empty)
cv2.createTrackbar('Val Max', 'TrackBar', 255, 255, empty)

hsv = cv2.cvtColor(block,cv2.COLOR_BGR2HSV)
while True:
    h_min = cv2.getTrackbarPos('Hue Min', 'TrackBar')
    h_max = cv2.getTrackbarPos('Hue Max', 'TrackBar')
    s_min = cv2.getTrackbarPos('Sat Min', 'TrackBar')
    s_max = cv2.getTrackbarPos('Sat Max', 'TrackBar')
    v_min = cv2.getTrackbarPos('Val Min', 'TrackBar')
    v_max = cv2.getTrackbarPos('Val Max', 'TrackBar')
    # print(h_min, h_max, s_min, s_max, v_min, v_max)
    
    lower = np.array([h_min, s_min, v_min])
    upper = np.array([h_max, s_max, v_max])
    
    mask = cv2.inRange(hsv, lower, upper)
    result = cv2.bitwise_and(block, block, mask=mask)

    cv2.imshow('block', block)
    # cv2.imshow('hsv', hsv)
    cv2.imshow('mask', mask)
    cv2.imshow('result', result)
    

    key = cv2.waitKey(1)
    if key == 27:
        break

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
hsv_boundary = {
    "red": np.array([[0, 150, 0], [20, 255, 255]], dtype=np.uint8),
    "blue": np.array([[0, 0, 0], [179, 70, 80]], dtype=np.uint8),
    "green": np.array([[38, 33, 0], [75, 165, 255]], dtype=np.uint8),
    "yellow": np.array([[20, 50, 110], [50, 255, 255]], dtype=np.uint8),
    "purple": np.array([[100, 0, 55], [160, 255, 160]], dtype=np.uint8)
}

In [ ]:
#抓圖片
img = cv2.imread("paper.jpg")
# img = cv2.imread("123.jpg")
# img = cv2.resize(img, (640, 480))

object_points = np.array([
    [0,0,0],#1
    [0,25,0],#2
    [25,25,0],#3
    [25,0,0],#4
    [0,0,25],#5
    [0,25,25],#6
    [25,25,25],#7
    [25,0,25],#8
], dtype=np.float32)
SS.reset()
text_offset = 0

# text_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
for image_points in CT.detect_parts(img, 2):
    # image_points = np.array(image_points, dtype=np.float32)
    for color_name, rgb in CT.get_color_text(img):
        image_points = np.float32(image_points)
        retval, rvec, tvec = cv2.solvePnP(object_points[:4], image_points, camera_matrix, dist_coeff)
        x = tvec[0]
        y = tvec[1]
        z = tvec[2]
        rotation_matrix, _ = cv2.Rodrigues(rvec)
        # 使用旋转矩阵计算欧拉角（roll-pitch-yaw 顺序）
        yaw = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
        pitch = np.arctan2(
            -rotation_matrix[2, 0],
            np.sqrt(rotation_matrix[2, 1] ** 2 + rotation_matrix[2, 2] ** 2),
        )
        roll = np.arctan2(rotation_matrix[2, 1], rotation_matrix[2, 2])
        # 将弧度转换为度数
        rz = np.degrees(yaw)
        ry = np.degrees(pitch)
        rx = np.degrees(roll)
        RotationZ = np.array([rz])
        text_loc_tvec = (5, 15 )
        text_loc_rvec = (5, 32 )
        text_loc_check = (400, 15 )
        # print(color_name)
        print(f"{x=}")
        print(f"{y=}")
        print(f"{rx=}")
        Save_2_environ.save_coordinate(color_name, x, y, RotationZ, 10)
        xyz_str = [f"{c}: {v[0]:.2f}" for c, v in zip("xyz", [x, y, z])]
        cv2.putText(img, f"{color_name} {', '.join(xyz_str)}", text_loc_tvec, cv2.FONT_HERSHEY_SIMPLEX, 0.5, rgb, 2)
        cv2.putText(img, f"Rotate Z: {rz:.1f},   Rotate Y: {ry:.1f},   Rotate X: {rx:.1f}", text_loc_rvec, cv2.FONT_HERSHEY_SIMPLEX, 0.5, rgb, 2)
        if -10 < x < 10 and -10 < y < 10:
            cv2.putText(img, "OK", text_loc_check, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        else:
            cv2.putText(img, "Moving", text_loc_check, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255) ,2)
        cv2.circle(img, (int(x[0]), int(y[0])), 5, (0, 0, 255), -1)

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def transform_matrix(x, y, z, rx, ry, rz):
    # 轉換為弧度
    rx, ry, rz = np.radians(rx), np.radians(ry), np.radians(rz)

    # 計算旋轉矩陣
    rotation_matrix = R.from_euler('xyz', [rx, ry, rz], degrees=False).as_matrix()

    # 構建轉移矩陣
    transformation_matrix = np.vstack([np.hstack([rotation_matrix, np.array([[x], [y], [z]])]),
                                      np.array([0, 0, 0, 1])])

    return transformation_matrix


In [ ]:
import numpy as np

def rotation_x(theta):
    return np.array([[1, 0, 0],
                     [0, np.cos(theta), -np.sin(theta)],
                     [0, np.sin(theta), np.cos(theta)]])

def rotation_y(theta):
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                     [0, 1, 0],
                     [-np.sin(theta), 0, np.cos(theta)]])

def rotation_z(theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])

def transform_matrix(x, y, z, rx, ry, rz):
    rx = np.radians(rx)
    ry = np.radians(ry)
    rz = np.radians(rz)

    rotation_matrix = np.dot(rotation_z(rz), np.dot(rotation_y(ry), rotation_x(rx)))
    translation_vector = np.array([[x], [y], [z]])

# 合併旋轉矩陣和平移向量
    transformation_matrix = np.vstack([np.hstack([rotation_matrix, translation_vector]),
                                      np.array([0, 0, 0, 1])])

    return transformation_matrix

In [2]:
lens = cv2.VideoCapture(0,cv2.CAP_DSHOW)

In [4]:
lens.release()

In [3]:
# object_points = np.array([
#     [-12.5,-12.5,0],#1
#     [-12.5,12.5,0],#2
#     [12.5,12.5,0],#3
    # [12.5,-12.5,0],#4
# ], dtype=np.float32)
object_points = np.array( #中心
    [
        [-25, -25, 0],  # 1
        [-25, 25, 0],  # 2
        [25, 25, 0],  # 3
        [25, -25, 0],  # 4
    ],
    dtype=np.float32,
)
# object_points = np.array([
#     [-12.5,-12.5,0],#1
#     [-12.5,12.5,0],#2
#     [12.5,12.5,0],#3
#     [12.5,-12.5,0],#4
# ], dtype=np.float32)
object_points = np.array(
    [
        [-25, -25, 0],  # 1
        [-25, 25, 0],  # 2
        [25, 25, 0],  # 3
        [25, -25, 0],  # 4
    ],
    dtype=np.float32,
)
Save_2_grip.reset()
offset_x = 0
offset_y = 0

while True:
    ret, img = lens.read()
    if not ret:
        break
    vertical_offset = 0
    for image_points in CT.detect_parts(img):
        for color_name, rgb in CT.get_color_text(img):
            image_points = np.float32(image_points)
            retval, rvec, tvec = cv2.solvePnP(object_points[:4], image_points, camera_matrix, dist_coeff)
            if not retval:
                break
            x = tvec[0] - offset_x
            y = tvec[1] - offset_y
            z = tvec[2]
            rotation_matrix, _ = cv2.Rodrigues(rvec)
            # 使用旋转矩阵计算欧拉角（roll-pitch-yaw 顺序）
            yaw = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
            pitch = np.arctan2(-rotation_matrix[2, 0],np.sqrt(rotation_matrix[2, 1] ** 2 + rotation_matrix[2, 2] ** 2))
            roll = np.arctan2(rotation_matrix[2, 1], rotation_matrix[2, 2])
            # 将弧度转换为度数
            rz = np.degrees(yaw)
            ry = np.degrees(pitch)
            rx = np.degrees(roll)
            RotationZ = np.array([rz])
            text_loc_tvec = (5, 15 + vertical_offset)
            text_loc_rvec = (5, 32 + vertical_offset)
            text_loc_check = (400, 15 + vertical_offset)

            Save_2_grip.save_coordinate_by_color(color_name, x, y,RotationZ, 10)
            xyz_str = [f"{c}: {v[0]:.2f}" for c, v in zip("xyz", [x, y, z])]
            cv2.putText(img, f"{color_name} {', '.join(xyz_str)}", text_loc_tvec, cv2.FONT_HERSHEY_SIMPLEX, 0.5, rgb, 2)
            cv2.putText(img, f"Rotate Z: {rz:.1f},   Rotate Y: {ry:.1f},   Rotate X: {rx:.1f}", text_loc_rvec, cv2.FONT_HERSHEY_SIMPLEX, 0.5, rgb, 2)
            if -10 < x < 10 and -10 < y < 10:
                cv2.putText(img, "OK", text_loc_check, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            else:
                cv2.putText(img, "Moving", text_loc_check, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255) ,2)
            vertical_offset += 34
            cv2.circle(img, (int(x[0]), int(y[0])), 5, (0, 0, 255), -1)

    else:
        pass
    cv2.imshow("test", img)
    key = cv2.waitKey(1)
    if key == 27:
        break
cv2.destroyAllWindows()

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R


def calculate_transform_matrix(initial_position, final_position):
    # 初始座標和移動後的座標
    p1 = np.array(initial_position)
    p2 = np.array(final_position)

    # 計算平移向量
    translation_vector = p2 - p1

    # 計算旋轉矩陣
    direction = p2 - p1
    axis = np.cross(np.array([1, 0, 0]), direction)
    angle = np.arccos(np.dot(np.array([1, 0, 0]), direction) / (np.linalg.norm(direction)))

    rotation_matrix = R.from_rotvec(axis * angle).as_matrix()

    # 構建轉移矩陣
    transformation_matrix = np.vstack([np.hstack([rotation_matrix, translation_vector.reshape(3, 1)]),
                                      np.array([0, 0, 0, 1])])

    return transformation_matrix
np.linalg.inv(transform_matrix(200,500,509,-180, 0, -60)) @ calculate_transform_matrix([0,0,509],[200,500,509])

In [ ]:
coordinates = Save_2_grip.get_coordinates_by_color(["green", "purple"])
for i in range(0, len(coordinates), 1):
    x = coordinates[i][0]
    y = coordinates[i][1]
    rx = coordinates[i][2]

In [ ]:
from ultralytics import YOLO
import robot.robot as bot
import socket
import pickle
import cv2
import numpy as np
import catcher as CT
import cube_detector.cube_detector as CD
import SaveSystem_by_environment
import SaveSystem_by_grip

model_part = YOLO("./cube_surface.pt")
model_region = YOLO("./cube.pt")

model = YOLO("./cube.pt")
surface_model = YOLO('./cube_surface.pt')
model_color = YOLO('./cube_color.pt')

with open('./hand_matrix/calibration.pkl', 'rb') as file:
    camera_matrix, dist_coeff = pickle.load(file)
with open("./fixedCam_matrix/MultiMatrix_fixed_640_480.npz","rb") as file:
    mtx = np.load(file)["camMatrix"]
    dist = np.load(file)["distCoef"]

CT = CT.block_detect(model_part, model_region)
detector = CD.CubeDetector(model, surface_model, model_color)
Save_2_environ = SaveSystem_by_environment.save_system()
Save_2_grip = SaveSystem_by_grip.save_system()

Save_2_environ.reset()
Save_2_grip.reset()


def draw_axis(img, corners, image_points):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(np.int16(image_points[0].ravel())), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(np.int16(image_points[1].ravel())), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(np.int16(image_points[2].ravel())), (0,0,255), 5)
    return img
def draw_xy_lines(img, image_points):
    img = cv2.line(img, tuple(np.int16(image_points[0].ravel())), tuple(np.int16(image_points[1].ravel())), (0,0,0), 5)
    img = cv2.line(img, tuple(np.int16(image_points[1].ravel())), tuple(np.int16(image_points[2].ravel())), (0,0,0), 5)
    return img
def draw_z_lines(img, image_points):
    img = cv2.line(img, tuple(np.int16(image_points[1].ravel())), tuple(np.int16(image_points[3].ravel())), (128,128,0), 5)
    return img

object_points = np.array( #中心
    [
        [-25, -25, 0],  # 1
        [-25, 25, 0],  # 2
        [25, 25, 0],  # 3
        [25, -25, 0],  # 4
    ],
    dtype=np.float32,
)

#-------------------抓環境座標----------------------
cap = cv2.VideoCapture(1,cv2.CAP_DSHOW)

fixed_to_table = np.float32(
    [[0, 1, 0, -650],
     [1, 0, 0, -250],
     [0, 0, -1, 860],
     [0, 0, 0, 1]])
_, table_rmtx, table_tvec, _, _, _, _  = cv2.decomposeProjectionMatrix(fixed_to_table[0:3,0:])

table_rvec, _ = cv2.Rodrigues(table_rmtx)
table_tvec = fixed_to_table[0:3,3]
print(table_tvec)   

# color = "yellow"
while True:
    cap_success, frame = cap.read()
    if cap_success:
        height = 480
        scale = height / frame.shape[0]
        frame = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
        #  將圖片丟入偵測器進行偵測
        result_img = detector.detect(frame, index=None, color="yellow",show_process_img=True)
        
        # 讀取抓到的角點座標
        for color_name in list(detector.cube_contour_outer.keys()):
            cube_imagePoints = detector.get_cube_sequence_imagePoints(color_name)
            if cube_imagePoints is None:
                cv2.imshow("result", frame)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
                continue
            # 依順序定出對應真實世界對應座標
            cube_objectPoints = np.array(
                [(0, 0, 0), (0, 50, 0), (50, 50, 0),
                (50, 0, 0), (50, 0, -50), (0, 0, -50)]
            )
            # 如果抓到六個點採用EPNP算法(較準確穩定)，四個點則是一般算法
            print(type(cube_imagePoints),'\n',f"{cube_imagePoints}")
            if cube_imagePoints.shape[0] == 6:
                PnP_success, cube_rvec, cube_tvec = cv2.solvePnP(
                    cube_objectPoints.astype(float), cube_imagePoints, mtx, dist, flags=cv2.SOLVEPNP_EPNP)
            elif cube_imagePoints.shape[0] == 4:
                PnP_success, cube_rvec, cube_tvec = cv2.solvePnP(
                    cube_objectPoints[:4].astype(float), cube_imagePoints, mtx, dist)
            #  若成功計算出來則計算出該方塊相對於桌面座標系的座標位置
            if PnP_success:
                R1, _ = cv2.Rodrigues(cube_rvec)  # 創建方塊到相機座標系的旋轉矩陣
                R2, _ = cv2.Rodrigues(table_rvec)  # 創建桌面到方塊座標系的旋轉矩陣
                T1 = np.eye(4)  # 創建4*4單位矩陣
                T2 = np.eye(4)
                T1[:3, :3], T1[:3, 3] = R1, cube_tvec.T  #  方塊座標系旋轉後平移到相機座標系 轉換矩陣
                T2[:3, :3], T2[:3, 3] = R2, table_tvec.T  # 桌面座標系旋轉後平移到相機座標系 轉換矩陣
                T2 = np.linalg.inv(T2)  # 取逆矩陣，變成 相譏座標系到桌面座標系 轉換矩陣
                # fixed_to_table = np.linalg.inv(fixed_to_table)  # 取逆矩陣，變成 相譏座標系到桌面座標系 轉換矩陣
                transform_matrix = T2 @ T1  # 結合二者，得到方塊座標系到桌面座標系的轉換矩陣
                transformed_point = (transform_matrix @ np.array([[25, 25, -25, 1]]).T)  # 傳入方塊正中央座標，即可得到方塊在桌面座標位置了
                # 输出结果
                print("物體在桌面座標系下的座標:\n", transformed_point)
            # xyz座標位置線繪製
            x, y, z = transformed_point[0:3].flatten()

            xyz_line_points = np.float32([[0, y, 0], [x, y, 0], [x, 0, 0], [x, y, z]]).reshape(-1, 3)
            xyz_line_points_img, _ = cv2.projectPoints(xyz_line_points, table_rvec, table_tvec, mtx, dist)
            result_img = draw_xy_lines(result_img, xyz_line_points_img)
            result_img = draw_z_lines(result_img, xyz_line_points_img)
            print(x, y)        # xyz座標軸繪製
            axis = np.float32([[25, 0, 0], [0, 25, 0], [0, 0, 25]]).reshape(-1, 3)
            axis_cube_img_points, _ = cv2.projectPoints(axis, cube_rvec, cube_tvec, mtx, dist)
            # axis_table_img_points, _ = cv2.projectPoints(axis, table_rvec, table_tvec, mtx, dist)
            result_img = draw_axis(result_img, cube_imagePoints.astype(int), axis_cube_img_points)
            # result_img = draw_axis(result_img, table_imagePoints.astype(int), axis_table_img_points)

        # 取得方塊輪廓計算方塊重心位置
        
        
            print(color_name)
            contour_outer = detector.get_cube_contour_outer(color_name)
            radius = int(0.08 * cv2.arcLength(contour_outer, True))
            print(radius)
            M = cv2.moments(contour_outer)
            if M["m00"] != 0:
                centroid_X = int(M["m10"] / M["m00"])  # 算形心x
                centroid_Y = int(M["m01"] / M["m00"])  # 算形心y
            # 繪製重心座標點
            result_img = cv2.circle(result_img, (centroid_X, centroid_Y), radius, (255, 0, 255), 2)

            # 繪製PNP計算所得座標點
            predict_cube_center_imagePoint = cv2.projectPoints(np.float32([x, y, z]), table_rvec, table_tvec, mtx, dist)
            predict_cube_center_imagePoint = predict_cube_center_imagePoint[0].astype(np.int16)

            cv2.putText(result_img, f"{x=:.1f}, {y=:.1f}, {z=:.1f}", (10, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (128, 0, 128), 2,)

            # 推算是否位置估計錯誤
            if (np.linalg.norm(np.array((centroid_X, centroid_Y)) - predict_cube_center_imagePoint) > radius):
                cv2.putText(result_img, f"Error", (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4,)
                result_img = cv2.circle(result_img, predict_cube_center_imagePoint.ravel(), 5, (0, 0, 255), -1)
            else:
                cv2.putText(result_img, f"Ok", (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4,)
                result_img = cv2.circle(result_img, predict_cube_center_imagePoint.ravel(), 5, (0, 255, 0), -1)
                # print(f"{type(color_name)=}")
                x,y,z = transformed_point[0:3]
                # print(f"{x=}")
                # print(f"{y=}")
                # print(f"{z=}")
                Save_2_environ.save_coordinate_by_color(color_name,x, y, z, 10)

        cv2.imshow("result", result_img)
        Key = cv2.waitKey(1)
        if Key == 27:
            break
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        if Save_2_environ.completed_Save == True:
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('./cube_surface.pt')

# Open the video file
video_path = 1
cap = cv2.VideoCapture(video_path,cv2.CAP_DSHOW)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame,conf=0.8)
        # if result.boxes.conf
        # Visualize the results on the frame
        annotated_frame = results[0].plot(labels=False)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Inference", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

In [51]:
grip_coor = Save_2_grip.get_coordinates_by_test("green")
print(grip_coor)

[array([      17.17,      -15.16,       38.67])]
